In [4]:
import pandas as pd
import numpy as np
import itertools
import pickle
import SEIR_full as mdl

In [4]:
# Age groups
A = {0: '0-4', 1: '5-9', 2: '10-19', 3: '20-29', 4: '30-39', 5: '40-49', 6: '50-59', 7: '60-69', 8: '70+'}

# Risk groups
R = {0: 'High', 1: 'Low'}

# Intervention groups
M = {0: 'Intervention', 1: 'Non-intervention'}

# region groups
G = {0: '11', 1: '11_betshemesh', 2: '21', 3: '22', 4: '23', 5: '24', 6: '29', 7: '31', 8: '32', 9: '41', 10: '42', 11: '43', 12: '44', 13: '51',
     14: '51_tlv', 15: '51_bb', 16: '61', 17: '62', 18: '62_arab', 19: '71'}

# All combination:
N = {i: group for i, group in enumerate(itertools.product(M.values(), G.values(), R.values(), A.values()))}

In [5]:
def get_oposite_dict(dic, keys):
    """The function gets a dict and new keys list and returns a dictionary in which keys as keys,
    and values are the keys from dic """
    res_dict = {}
    
    for new_key in keys:
        new_val_lst = []
        for old_k, old_v in dic.items():
            if all(i in old_v for i in new_key):
                new_val_lst.append(old_k)
        if len(new_key) == 1:
            res_dict[new_key[0]] = new_val_lst
        else:
            res_dict[new_key] = new_val_lst
    return res_dict

def expand_partial_array(mapping_dic, array_to_expand):
    """The function gets mapping_dic - indeces to assign in the expanded array (with key granularity based on
    array_to_expand), and array_to_expand - the expanded array's values will be based on this array. Returns 
    and expanded array shape (len(N),1) based on the array_to_expand"""
    
    # Creating dictionary that maps the indices in the array to expand
#     small_mapping_dic = {k:v[0] for k,v in mapping_dic.items()}
    
    # Assinging values to the full array
    full_array = np.zeros(len(N))
    for i,k in enumerate(mapping_dic.keys()):
        full_array[mapping_dic[k]] = array_to_expand[i]
    
    return full_array

In [29]:
# opposite deict:
region_age = get_oposite_dict(N, list(itertools.product(G.values(), A.values())))
age = get_oposite_dict(N, [['0-4'], ['5-9'], ['10-19'], ['20-29'], ['30-39'], ['40-49'], ['50-59'], ['60-69'], ['70+']])
region_risk_age = get_oposite_dict(N, list(itertools.product(G.values(), R.values(), A.values())))
risk_age = get_oposite_dict(N, list(itertools.product(R.values(), A.values())))

In [95]:
mdl.N

{0: ('Intervention', '11', 'High', '0-4'),
 1: ('Intervention', '11', 'High', '5-9'),
 2: ('Intervention', '11', 'High', '10-19'),
 3: ('Intervention', '11', 'High', '20-29'),
 4: ('Intervention', '11', 'High', '30-39'),
 5: ('Intervention', '11', 'High', '40-49'),
 6: ('Intervention', '11', 'High', '50-59'),
 7: ('Intervention', '11', 'High', '60-69'),
 8: ('Intervention', '11', 'High', '70+'),
 9: ('Intervention', '11', 'Low', '0-4'),
 10: ('Intervention', '11', 'Low', '5-9'),
 11: ('Intervention', '11', 'Low', '10-19'),
 12: ('Intervention', '11', 'Low', '20-29'),
 13: ('Intervention', '11', 'Low', '30-39'),
 14: ('Intervention', '11', 'Low', '40-49'),
 15: ('Intervention', '11', 'Low', '50-59'),
 16: ('Intervention', '11', 'Low', '60-69'),
 17: ('Intervention', '11', 'Low', '70+'),
 18: ('Intervention', '11_betshemesh', 'High', '0-4'),
 19: ('Intervention', '11_betshemesh', 'High', '5-9'),
 20: ('Intervention', '11_betshemesh', 'High', '10-19'),
 21: ('Intervention', '11_betshemesh

# Asymptomatic

In [14]:
asymp = pd.read_csv('../Data/asymptomatic proportions.csv', index_col=0)
asymp.head()

,Scenario 1,Scenario 2,Scenario 3,Unnamed: 4
Age,,,,
0-4,0.957467,0.765973,0.612779,NaN
5-9,0.957467,0.765973,0.612779,NaN
10-19,0.972167,0.777734,0.622187,NaN
20-29,0.921424,0.737139,0.589711,NaN
30-39,0.899804,0.719843,0.575874,NaN


In [16]:
# asymptomatic with risk group, high risk with 0
f_init = np.zeros(len(list(itertools.product(mdl.R.values(), mdl.A.values()))))
f_init1 = f_init.copy()
f_init1[9:] = asymp['Scenario 1'].values[:-1]

f_init2 = f_init.copy()
f_init2[9:] = asymp['Scenario 2'].values[:-1]

f_init3 = f_init.copy()
f_init3[9:] = asymp['Scenario 3'].values[:-1]

In [30]:
f0_full = {} # dict that contains the possible scenarios
f_sce1 = expand_partial_array(risk_age, f_init1)
f_sce2 = expand_partial_array(risk_age, f_init2)
f_sce3 = expand_partial_array(risk_age, f_init3)

In [31]:
f0_full['Scenario1'] = f_sce1
f0_full['Scenario2'] = f_sce2
f0_full['Scenario3'] = f_sce3

In [34]:
# Save
with open('../Data/parameters/f0_full.pickle', 'wb') as handle:
    pickle.dump(f0_full, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Initial illness

In [137]:
# Age dist. positive specimens
age_dist = {'0-4':0.02, '5-9':0.02, '10-19':0.11, '20-29':0.23, '30-39':0.15, '40-49':0.14, '50-59':0.14, '60-69':0.11,
           '70+':0.08}

In [80]:
# Age distribution:
with open('../Data/demograph/init_pop.pickle', 'rb') as f:
    pop_dist = pickle.load(f)
    
# risk distribution by age:
risk_dist = pd.read_csv('../Data/population size.csv')

init_I_dis = pd.read_csv('../Data/init_i_italy.csv')['proportion'].values[:-1]

eps_t = {}

In [5]:
init_I_dis.sum()

0.9999999999999999

In [151]:
# Scenario 1
init_I_IL1 = (491./(1-(f_init1 * risk_dist['pop size'].values).sum())) / 9136000.
init_I_dis1 = init_I_dis * init_I_IL1

# Scenario 2
init_I_IL2 = (491./(1-(f_init2 * risk_dist['pop size'].values).sum())) / 9136000.
init_I_dis2 = init_I_dis * init_I_IL2

# Scenario 3
init_I_IL3 = (491./(1-(f_init3 * risk_dist['pop size'].values).sum())) / 9136000.
init_I_dis3 = init_I_dis * init_I_IL3

In [64]:
eps_sc1 = []
for i in range(1000):
    eps_sc1.append(init_I_dis1[i] * pop_dist if i < len(init_I_dis1) else 0)

eps_sc2 = []
for i in range(1000):
    eps_sc2.append(init_I_dis2[i] * pop_dist if i < len(init_I_dis2) else 0)

eps_sc3 = []
for i in range(1000):
    eps_sc3.append(init_I_dis3[i] * pop_dist if i < len(init_I_dis3) else 0)
    
    
eps_t['Scenario1'] = eps_sc1
eps_t['Scenario2'] = eps_sc2
eps_t['Scenario3'] = eps_sc3

In [65]:
# Save
with open('../Data/parameters/eps_dict.pickle', 'wb') as handle:
    pickle.dump(eps_t, handle, protocol=pickle.HIGHEST_PROTOCOL)

## eps by region proportion

In [133]:
# Loading data
region_prop = pd.read_csv('../Data/Israel/positive_prop_region_2703.csv', index_col=0)['cases_prop'].copy()
risk_prop = pd.read_csv('../Data/demograph/risk_dist.csv', index_col=0)['risk'].copy()
print(region_prop.head())
print()
print(risk_prop.head())

sub_district_id
11               0.137496
11_betshemesh    0.012624
21               0.010918
22               0.006482
23               0.037189
Name: cases_prop, dtype: float64

Age
0-4      0.050
5-9      0.106
10-19    0.106
20-29    0.149
30-39    0.149
Name: risk, dtype: float64


In [97]:
eps_t_region = {}

In [152]:
for sc,init_I in [('Scenario1',init_I_dis1),('Scenario2',init_I_dis2),('Scenario3',init_I_dis3)]:
    eps_temp = []
    for t in range(1000):
        if t < len(init_I):
            # empty array for day t
            day_vec = np.zeros(len(mdl.N))
            # fill in the array, zero for intervention groups
            for key in mdl.N.keys():
                if mdl.N[key][0] == 'Intervention':
                    day_vec[key] = 0
                else:
                    day_vec[key] = init_I[t] * region_prop[mdl.N[key][1]] * age_dist[mdl.N[key][3]] * \
                    (risk_prop[mdl.N[key][3]]**(1 - (mdl.N[key][2] == 'Low'))) * \
                    ((1 - risk_prop[mdl.N[key][3]])**(mdl.N[key][2] == 'Low'))
            eps_temp.append(day_vec)
        else:
             eps_temp.append(0.0)
        
        eps_t_region[sc] = eps_temp


In [168]:
# save eps:
with open('Data/parameters/eps_by_region.pickle', 'wb') as handle:
    pickle.dump(eps_t_region, handle, protocol=pickle.HIGHEST_PROTOCOL)  

## Smoothed eps by region

In [13]:
init_eps = pd.read_csv('../Data/Israel/IL_initial_eps.csv', header=None).values[:,0].copy()
init_eps

array([  0.78180071,   2.94105921,   5.84488904,   8.5253491 ,
        12.02483762,  17.497441  ,  24.38473153,  33.09622265,
        41.99385742,  39.83459892,  75.79742365,  93.14595037,
       114.9991361 , 143.6279275 , 172.5917768 , 219.0158361 ,
       274.5609032 , 341.4606921 ])

In [51]:
# Scenario 1
init_eps_asymp1 = (init_eps/(1-(f_init1 * risk_dist['pop size'].values).sum())) / 9136000.

# Scenario 2
init_eps_asymp2 = (init_eps/(1-(f_init2 * risk_dist['pop size'].values).sum())) / 9136000.


# Scenario 3
init_eps_asymp3 = (init_eps/(1-(f_init3 * risk_dist['pop size'].values).sum())) / 9136000.


In [26]:
f_init3

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.6127786 ,
       0.6127786 , 0.62218712, 0.58971125, 0.57587446, 0.59021178,
       0.56415614, 0.54046208, 0.14933333])

In [53]:
init_eps_asymp3.sum()*9136000

3003.15972721764

In [54]:
def fill_eps_day(init_eps, dic=mdl.N, age_)

{0: '11',
 1: '11_betshemesh',
 2: '21',
 3: '22',
 4: '23',
 5: '24',
 6: '29',
 7: '31',
 8: '32',
 9: '41',
 10: '42',
 11: '43',
 12: '44',
 13: '51',
 14: '51_tlv',
 15: '51_bb',
 16: '61',
 17: '62',
 18: '62_arab',
 19: '71'}

In [ ]:
eps_reg = []


# Hospitalizations

In [35]:
hosp_init = pd.read_csv('../Data/hospitalizations.csv')
hosp_init

,Age,risk,pr_hospital
0,0-4,High,0.029022
1,5-9,High,0.025129
2,10-19,High,0.025129
3,20-29,High,0.045566
4,30-39,High,0.113915
5,40-49,High,0.136698
6,50-59,High,0.163551
7,60-69,High,0.207688
8,70+,High,0.184739
9,0-4,Low,0.006894


In [36]:
hosp = expand_partial_array(risk_age, hosp_init['pr_hospital'].values)

In [38]:
# Save
with open('../Data/parameters/hospitalization.pickle', 'wb') as handle:
    pickle.dump(hosp, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Calibration contact matrix

In [1]:
import SEIR_full as mdl

In [11]:
mdl.C_calibration['work_non'][-1]

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.37346519e+00, 6.13787276e+00, ...,
        3.74307514e-04, 2.80915810e-05, 2.85544705e-08],
       [0.00000000e+00, 4.74965983e+00, 2.49011976e+01, ...,
        1.98828583e-03, 2.00107399e-04, 1.09157596e-07],
       ...,
       [0.00000000e+00, 1.52767089e-04, 5.19119040e-04, ...,
        1.27647753e+02, 8.53740114e+00, 4.88476496e-03],
       [3.21804854e-09, 1.27495193e-05, 6.41211593e-05, ...,
        1.51984520e+01, 1.18440308e+00, 8.16887248e-03],
       [0.00000000e+00, 5.37320192e-09, 5.12545939e-08, ...,
        3.64201110e-03, 3.65020647e-03, 3.91885098e-03]])

Important : here no intervention mean the course of interventions made by government

In [9]:
full_mtx_home = pd.read_csv('../Data/base_contact_mtx/full_home_mtx.csv', index_col=[0,1],header=[0, 1])
full_mtx_work = {'routine':pd.read_csv('../Data/base_contact_mtx/full_work_mtx_routine.csv', index_col=[0,1],header=[0, 1]),
                        'no_school':pd.read_csv('../Data/base_contact_mtx/full_work_mtx_no_school.csv', index_col=[0,1],header=[0, 1]),
                        'no_work':pd.read_csv('../Data/base_contact_mtx/full_work_mtx_no_work.csv', index_col=[0,1],header=[0, 1])}

full_mtx_leisure = {'routine':pd.read_csv('../Data/base_contact_mtx/full_leisure_mtx_routine.csv', index_col=[0,1],header=[0, 1]),
                        'no_school':pd.read_csv('../Data/base_contact_mtx/full_leisure_mtx_no_school.csv', index_col=[0,1],header=[0, 1]),
                        'no_work':pd.read_csv('../Data/base_contact_mtx/full_leisure_mtx_no_work.csv', index_col=[0,1],header=[0, 1])}

In [10]:
full_mtx_work['no_work']

11                                            \
age                  0-4         10-19         20-29         30-39   
area age                                                             
11   0-4    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
     10-19  0.000000e+00  4.462206e+00  6.262414e+00  6.088536e+00   
     20-29  0.000000e+00  4.846033e+00  2.540646e+01  3.027929e+01   
     30-39  0.000000e+00  3.863133e+00  1.904872e+01  4.112174e+01   
     40-49  0.000000e+00  3.597359e+00  1.633521e+01  3.492237e+01   
...                  ...           ...           ...           ...   
71   40-49  0.000000e+00  1.158186e-04  5.259196e-04  1.124342e-03   
     5-9    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
     50-59  0.000000e+00  1.061938e-04  3.608581e-04  8.149323e-04   
     60-69  2.236980e-09  8.862645e-06  4.457290e-05  9.314097e-05   
     70+    0.000000e+00  3.735104e-09  3.562890e-08  7.189281e-08   

                                                                    \
age                40-49           5-9         50-59         60-69   
area age                                                             
11   0-4    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
     10-19  7.613842e+00  0.000000e+00  3.581205e+00  2.350574e-01   
     20-29  2.878416e+01  0.000000e+00  1.902302e+01  1.674407e+00   
     30-39  4.454208e+01  0.000000e+00  2.510209e+01  1.563414e+00   
     40-49  4.928711e+01  0.000000e+00  2.960897e+01  1.691857e+00   
...                  ...           ...           ...           ...   
71   40-49  1.586821e-03  0.000000e+00  9.532742e-04  5.447010e-05   
     5-9    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
     50-59  1.298508e-03  0.000000e+00  1.029947e-03  6.024550e-05   
     60-69  1.368005e-04  1.094925e-09  1.226313e-04  8.357925e-06   
     70+    3.898215e-08  0.000000e+00  2.938618e-08  2.575825e-08   

                         11_betshemesh  ...       62_arab        71            \
age                  70+           0-4  ...           70+       0-4     10-19   
area age                                ...                                     
11   0-4    1.691741e-03  0.000000e+00  ...  0.000000e+00  0.000000  0.000000   
     10-19  1.517325e-04  0.000000e+00  ...  4.635329e-07  0.000000  0.000365   
     20-29  5.800408e-04  0.000000e+00  ...  1.771986e-06  0.000000  0.000397   
     30-39  4.340703e-04  0.000000e+00  ...  1.326056e-06  0.000000  0.000316   
     40-49  7.797899e-04  0.000000e+00  ...  2.382207e-06  0.000000  0.000295   
...                  ...           ...  ...           ...       ...       ...   
71   40-49  2.510569e-08  0.000000e+00  ...  2.229551e-06  0.000000  7.663010   
     5-9    0.000000e+00  0.000000e+00  ...  0.000000e+00  0.000000  0.000000   
     50-59  2.189018e-08  0.000000e+00  ...  1.943993e-06  0.000000  7.026200   
     60-69  3.660731e-08  4.289181e-08  ...  3.250972e-06  0.000122  0.586387   
     70+    1.756162e-08  0.000000e+00  ...  1.559588e-06  0.000000  0.000247   

                                                                             \
age             20-29      30-39       40-49       5-9      50-59     60-69   
area age                                                                      
11   0-4     0.000000   0.000000    0.000000  0.000000   0.000000  0.000000   
     10-19   0.000618   0.000501    0.000674  0.000000   0.000382  0.000029   
     20-29   0.002508   0.002493    0.002546  0.000000   0.002029  0.000204   
     30-39   0.001881   0.003385    0.003940  0.000000   0.002677  0.000191   
     40-49   0.001613   0.002875    0.004360  0.000000   0.003158  0.000206   
...               ...        ...         ...       ...        ...       ...   
71   40-49  41.946648  74.776829  113.402041  0.000000  82.126915  5.365729   
     5-9     0.000000   0.000000    0.000000  0.000000   0.000000  0.006650   
     50-59  28.781569  54.198883   92.797777  0.000000  

In [12]:
C_calibration = {}

In [13]:
home_no_inter = []
work_no_inter = []
leis_no_inter = []

home_inter = []
work_inter = []
leis_inter = []

In [14]:
# no intervation are null groups
for i in range(1000):
    home_no_inter.append(np.zeros((180,180)))
    work_no_inter.append(np.zeros((180,180)))
    leis_no_inter.append(np.zeros((180,180)))

In [15]:
# Intervantion
# first days of routine from feb 21st
for i in range(9+14):
    home_inter.append(full_mtx_home.values)
    work_inter.append(full_mtx_work['routine'].values)
    leis_inter.append(full_mtx_leisure['routine'].values)    

# first days of no school from march 14th
for i in range(2):
    home_inter.append(full_mtx_home.values)
    work_inter.append(full_mtx_work['no_school'].values)
    leis_inter.append(full_mtx_leisure['no_school'].values)   

# without school and work
for i in range(1000-2-9-14):
    home_inter.append(full_mtx_home.values)
    work_inter.append(full_mtx_work['no_work'].values)
    leis_inter.append(full_mtx_leisure['no_work'].values) 

In [16]:
C_calibration['home_inter'] = home_no_inter
C_calibration['work_inter'] = work_no_inter
C_calibration['leisure_inter'] = leis_no_inter
C_calibration['home_non'] = home_inter
C_calibration['work_non'] = work_inter
C_calibration['leisure_non'] = leis_inter

In [17]:
# Save
with open('../Data/parameters/C_calibration.pickle', 'wb') as handle:
    pickle.dump(C_calibration, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Haredi vector

In [2]:
# Loading raw data
hared_dis = pd.read_csv('../Data/demograph/religion_dis.csv', index_col=0)['haredi'].copy()
hared_dis.head()

sub_district_id
11               0.22320
11_betshemesh    0.50000
21               0.13410
22               0.05632
23               0.00000
Name: haredi, dtype: float64

In [5]:
# Creating model orthodox dist. and save it as pickle
model_orthodox_dis = np.zeros(len(mdl.GA))
for i in mdl.GA.keys():
    model_orthodox_dis[i] = hared_dis[mdl.GA[i][0]]
    
with open('Data/parameters/orthodox_dist.pickle', 'wb') as handle:
    pickle.dump(model_orthodox_dis, handle, protocol=pickle.HIGHEST_PROTOCOL)  